<a href="https://colab.research.google.com/github/mmaguero/diploma_fpuna_nlp_ia/blob/master/2025/final_project_guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparación y unificación del Diccionario Guaraní-Español y Español-Guaraní para Fine-tuning

En esta sección se procesan los diccionarios estructurados de ambos sentidos (español-guaraní y guaraní-español) y se unifican en un solo archivo para su uso en tareas de PLN y entrenamiento de modelos. No se generan archivos intermedios, solo el archivo final unificado.

Repositorio de este proyecto:
https://github.com/walterortiz07/proyecto_diplomado_pln

Instalamos las librerias

In [2]:
!pip install pdfplumber pandas numpy



[notice] A new release of pip available: 22.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Importamos las librerías:

In [5]:
## Para datos
import pandas as pd
import numpy as np

import pdfplumber
import re


Abrimos el data-set como un dataframe: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

Entrada: Identificador del archivo.

Salida: Una tabla DataFrame de Pandas.

In [7]:
# Inspeccionar fontnames únicos en las páginas relevantes para detectar negrita
pdf_path = 'Diccionario Guaraní-Español Español-Guaraní.pdf'
from collections import Counter

def print_unique_fontnames(pdf_path, start_page, end_page):
    fontnames = Counter()
    with pdfplumber.open(pdf_path) as pdf:
        for i in range(start_page-1, end_page):
            page = pdf.pages[i]
            words = page.extract_words(use_text_flow=True, keep_blank_chars=True, extra_attrs=["fontname"])
            for w in words:
                fontnames[w.get('fontname', 'None')] += 1
    print('Fontnames únicos y su frecuencia:')
    for font, count in fontnames.most_common():
        print(f'{font}: {count}')

# Cambia el rango según el diccionario que quieras analizar
print_unique_fontnames(pdf_path, 13, 85)  # Prueba con las primeras páginas del diccionario Guaraní-Español

Fontnames únicos y su frecuencia:
UNPVBK+ArialMT: 8096
QMVCXS+Arial-BoldMT: 3921
CJUWJU+Arial-BoldMT: 639
VEGBPI+ArialMT: 344
ZUIJTA+Alegreya-Regular: 73
TEBKNM+SuezOne-Regular: 3
UDQQBK+SuezOne-Regular: 2


### Extracción paso a paso del diccionario a CSV
A continuación, se muestra el proceso detallado y dividido en celdas para extraer las entradas del diccionario desde el PDF, construir el DataFrame y guardar los archivos CSV. El resultado final será idéntico al método anterior, pero cada paso será explicado y ejecutado por separado.

In [8]:
# 1. Extraer palabras y atributos de las páginas relevantes del PDF

def extract_words_with_attrs(pdf_path, start_page, end_page):
    all_words = []
    with pdfplumber.open(pdf_path) as pdf:
        for i in range(start_page-1, end_page):
            page = pdf.pages[i]
            width = page.width
            height = page.height
            mid = width / 2
            for col_idx, bbox in enumerate([(0, 0, mid, height), (mid, 0, width, height)]):
                col = page.within_bbox(bbox)
                if not col:
                    continue
                words = col.extract_words(use_text_flow=True, keep_blank_chars=True, extra_attrs=["fontname", "top"])
                words = [dict(w, page=i+1, col=col_idx+1) for w in words if not re.fullmatch(r'\d+', w["text"])]
                all_words.extend(words)
    return all_words

# Ejemplo: extraer palabras del diccionario Guaraní-Español (páginas 13 a 85)
words_guarani_espanol = extract_words_with_attrs(pdf_path, 13, 85)
# Y del diccionario Español-Guaraní (páginas 87 a 213)
words_espanol_guarani = extract_words_with_attrs(pdf_path, 87, 213)

print(f"Palabras extraídas Guaraní-Español: {len(words_guarani_espanol)}")
print(f"Palabras extraídas Español-Guaraní: {len(words_espanol_guarani)}")

Palabras extraídas Guaraní-Español: 13000
Palabras extraídas Español-Guaraní: 20548


#### Paso 2: Detectar inicios de línea y marcar palabras iniciales y negrita

Ahora, para cada columna, detectamos los saltos de línea (por cambios en la coordenada 'top') y marcamos qué palabras son iniciales y cuáles están en negrita.

In [9]:
# 2. Marcar palabras iniciales y negrita en cada columna

def mark_initial_and_bold(words):
    df = pd.DataFrame(words)
    df['is_bold'] = df['fontname'].apply(lambda f: f is not None and f.endswith('Arial-BoldMT'))
    df['is_initial'] = False
    for (page, col), group in df.groupby(['page', 'col']):
        tops = group['top'].values
        idxs = group.index.values
        line_starts = [0]
        threshold = 3
        for j in range(1, len(tops)):
            if abs(tops[j] - tops[j-1]) > threshold:
                line_starts.append(j)
        df.loc[idxs[line_starts], 'is_initial'] = True
    return df

df_guarani_espanol = mark_initial_and_bold(words_guarani_espanol)
df_espanol_guarani = mark_initial_and_bold(words_espanol_guarani)

df_guarani_espanol.head()

,text,x0,x1,top,doctop,bottom,upright,height,width,direction,fontname,page,col,is_bold,is_initial
0,a.,56.6929,65.9769,181.6169,10284.2969,192.6169,True,11.0,9.2840,ltr,QMVCXS+Arial-BoldMT,13,1,True,True
1,Vocal que se pronuncia igual que en español.,65.9756,293.6382,181.0559,10283.7359,192.0559,True,11.0,227.6626,ltr,UNPVBK+ArialMT,13,1,False,False
2,2. Pref. a. v. de 1ª. Per. sin. para verbos pr...,56.6929,290.5298,194.2560,10296.9360,205.2560,True,11.0,233.8369,ltr,UNPVBK+ArialMT,13,1,False,True
3,ã.,56.6929,65.9775,213.6862,10316.3662,224.6862,True,11.0,9.2846,ltr,QMVCXS+Arial-BoldMT,13,1,True,True
4,Vocal que se pronuncia igual que la,65.9756,257.2546,213.1252,10315.8052,224.1252,True,11.0,191.2790,ltr,UNPVBK+ArialMT,13,1,False,False


#### Paso 3: Reconstruir las entradas (headword y traducción) a partir de las palabras marcadas

Ahora, reconstruimos cada entrada del diccionario: solo cuando una palabra es inicial y negrita, se inicia una nueva entrada; el resto se concatena según las reglas.

In [10]:
# 3. Reconstruir entradas: headword (palabra en negrita inicial) y traducción

def build_entries(df):
    entries = []
    line = ''
    bold_headword = None
    for _, row in df.iterrows():
        word = row['text']
        is_initial = row['is_initial']
        is_bold = row['is_bold']
        if is_initial:
            if is_bold:
                if line.strip() and bold_headword:
                    palabra = bold_headword.strip().rstrip('.')
                    traduccion = line.strip()[len(bold_headword):].lstrip(' .:;-')
                    entries.append({'palabra': palabra, 'traduccion': traduccion})
                line = word
                bold_headword = word
            else:
                if line.strip():
                    line += ' ' + word
                else:
                    line = word
        else:
            line += ' ' + word
    if line.strip() and bold_headword:
        palabra = bold_headword.strip().rstrip('.')
        traduccion = line.strip()[len(bold_headword):].lstrip(' .:;-')
        entries.append({'palabra': palabra, 'traduccion': traduccion})
    return pd.DataFrame(entries)

# Aplicar a ambos diccionarios
df_entries_guarani_espanol = build_entries(df_guarani_espanol)
df_entries_espanol_guarani = build_entries(df_espanol_guarani)

df_entries_guarani_espanol.head()

,palabra,traduccion
0,a,Vocal que se pronuncia igual que en español. ...
1,ã,Vocal que se pronuncia igual que la a pero ...
2,‘a,"s. Fruto, fruta. 2. Caída. 3. Huevo."
3,‘ã,s. Ausencia. 2. Abrigo. 3. Sombra.
4,ache,"s. Parcialidad indígena guaraní, guayaki."


#### Paso 4: Creamos la carpeta archivos

In [11]:
import os

# Asegurar la existencia de la carpeta donde se guardará el archivo final unificado
os.makedirs('archivos', exist_ok=True)
print('Carpeta "archivos" lista para guardar el archivo final unificado.')

Carpeta "archivos" lista para guardar el archivo final unificado.


In [12]:
# --- Extracción robusta de todas las abreviaturas para ambos diccionarios (actualizado con nuevas abreviaturas y limpieza de numeraciones residuales) ---
import pandas as pd
import re

# Listas de abreviaturas
TIPOS_ETIQUETA = ['neol.', 'arc.', 'h.']
CATEGORIAS_GRAMATICALES = [
    'adj.', 'tr.', 'adv.', 'prep.', 's.', 'v.', 'participio pasivo', 'participio activo',
    'verbo transitivo', 'verbo intransitivo', 'adjetivo', 'sustantivo', 'pron.', 'conj.', 'interj.',
    'art.', 'num.', 'part.', 'pref.', 'suf.', 'f.', 'm.', 'ta.', 'Tr.', 'prnl.', 'ra.', 'com.', 'int.', 'va.',
    'da.', 'intr.', 'sa.', 'y f.', 'm y f.', 'va.', 'intrans.', 'trans.', 'exp.', 'pl.', 'dem.', 'sing.', 'rel.', 'int.', 'com.', 'adj', 'intr', 'tr', 'f', 'm', 'ta', 'da', 'sa', 'va', 'ra', 'com', 'int', 'pl', 'dem', 'sing', 'rel', 'exp', 'prnl', 'intrans', 'trans', 'gram.',
    # Nuevas abreviaturas encontradas:
    'atr.', 'adv .', 'ind.', 'neg.', 'pr.'
]
# Unir ambas listas para búsqueda general
ALL_ABBR = TIPOS_ETIQUETA + CATEGORIAS_GRAMATICALES
# Ordenar por longitud descendente para evitar capturas parciales
ALL_ABBR = sorted(set(ALL_ABBR), key=len, reverse=True)
ABBR_REGEX = r'|'.join([re.escape(a) for a in ALL_ABBR])

# Función robusta de extracción de todas las abreviaturas
def extract_labels_and_clean(text):
    # Buscar todas las abreviaturas en cualquier parte del texto
    abbrs = re.findall(rf'({ABBR_REGEX})', text)
    tipo_etiqueta = []
    categoria_gramatical = []
    for abbr in abbrs:
        if abbr in TIPOS_ETIQUETA:
            tipo_etiqueta.append(abbr)
        elif abbr in CATEGORIAS_GRAMATICALES:
            categoria_gramatical.append(abbr)
    # Eliminar todas las abreviaturas encontradas del texto
    for abbr in abbrs:
        text = re.sub(rf'\b{re.escape(abbr)}[\s,.]*', '', text)
    text = text.lstrip('. ').strip()
    # Quitar punto final si existe
    text = text.rstrip('.')
    # Eliminar numeraciones residuales (ej: '2.', '3.', etc. al inicio o en medio)
    text = re.sub(r'(\b\d+\.)', '', text).strip()
    # Eliminar espacios dobles generados
    text = re.sub(r'\s+', ' ', text)
    return ' '.join(tipo_etiqueta), ' '.join(categoria_gramatical), text

def split_and_expand(row, sentido):
    if sentido == 'ge':
        palabra_col = 'palabra' if 'palabra' in row else 'palabra_guarani'
        palabra = row.get(palabra_col, '')
    else:
        palabra_col = 'palabra' if 'palabra' in row else 'palabra_espanol'
        palabra = row.get(palabra_col, '')
    output = str(row['traduccion']) if 'traduccion' in row else str(row.get('output', ''))
    output = output if not pd.isnull(output) else ''
    # Separar por número (2., 3., ...)
    partes = re.split(r'(?<=\.)\s*(\d+\.)\s*', output)
    traducciones = []
    if partes:
        traducciones.append(partes[0].strip())
        for i in range(1, len(partes), 2):
            traducciones.append(partes[i+1].strip() if i+1 < len(partes) else '')
    else:
        traducciones = [output.strip()]
    filas = []
    for trad in traducciones:
        subtrads = [t.strip() for t in trad.split(',') if t.strip()]
        for subtrad in subtrads:
            tipo_etiqueta, categoria_gramatical, subtrad_limpia = extract_labels_and_clean(subtrad)
            if sentido == 'ge':
                fila = {
                    'palabra_guarani': palabra,
                    'tipo_etiqueta': tipo_etiqueta,
                    'categoria_gramatical': categoria_gramatical,
                    'traduccion_limpia': subtrad_limpia
                }
            else:
                fila = {
                    'palabra_espanol': palabra,
                    'tipo_etiqueta': tipo_etiqueta,
                    'categoria_gramatical': categoria_gramatical,
                    'traduccion_limpia': subtrad_limpia
                }
            filas.append(fila)
    return filas

# Procesar guaraní-español
ge_expandidas = []
for _, row in df_entries_guarani_espanol.iterrows():
    ge_expandidas.extend(split_and_expand(row, 'ge'))
df_estructurado_ge = pd.DataFrame(ge_expandidas)
cols_ge = ['palabra_guarani', 'tipo_etiqueta', 'categoria_gramatical', 'traduccion_limpia']
df_estructurado_ge = df_estructurado_ge[cols_ge]
# Eliminar filas vacías en traduccion_limpia
df_estructurado_ge = df_estructurado_ge[df_estructurado_ge['traduccion_limpia'].str.strip() != '']
df_estructurado_ge.to_csv('archivos/diccionario_guarani_espanol_estructurado.tsv', sep='\t', index=False, encoding='utf-8')
print('Archivo estructurado guardado: archivos/diccionario_guarani_espanol_estructurado.tsv')

# Procesar español-guaraní
eg_expandidas = []
for _, row in df_entries_espanol_guarani.iterrows():
    eg_expandidas.extend(split_and_expand(row, 'eg'))
df_estructurado_eg = pd.DataFrame(eg_expandidas)
cols_eg = ['palabra_espanol', 'tipo_etiqueta', 'categoria_gramatical', 'traduccion_limpia']
df_estructurado_eg = df_estructurado_eg[cols_eg]
# Eliminar filas vacías en traduccion_limpia
df_estructurado_eg = df_estructurado_eg[df_estructurado_eg['traduccion_limpia'].str.strip() != '']
df_estructurado_eg.to_csv('archivos/diccionario_espanol_guarani_estructurado.tsv', sep='\t', index=False, encoding='utf-8')
print('Archivo estructurado guardado: archivos/diccionario_espanol_guarani_estructurado.tsv')

from IPython.display import display
print('Ejemplo guaraní-español:')
display(df_estructurado_ge.sample(10))
print('Ejemplo español-guaraní:')
display(df_estructurado_eg.sample(10))


Archivo estructurado guardado: archivos/diccionario_guarani_espanol_estructurado.tsv
Archivo estructurado guardado: archivos/diccionario_espanol_guarani_estructurado.tsv
Ejemplo guaraní-español:


,palabra_guarani,tipo_etiqueta,categoria_gramatical,traduccion_limpia
4561,kopi,,m ta,desmontar
519,apykatymói,,s.,Sillón
3733,jokapyre,,adj. ra,Quebrado
8420,poyhu,,,sospecha
10526,teka,,,t
4024,ka’aguygua,,sa va,lvaje
11987,ymaguare,,s. da,Antigüedad
7527,oñoñe’ẽ,,,acuerdo
3429,jerereko,,s. ta,Potestad
1042,gua,,ra,natural de


Ejemplo español-guaraní:


,palabra_espanol,tipo_etiqueta,categoria_gramatical,traduccion_limpia
11579,probar,,tr.,Ha’ã
11219,positivamente,,adv.,Añetehápe
2114,ceniza,,,kusugue
12624,reencontrar,,va,ñuvaitĩ jey
15496,"vergonzoso, sa",,m va.,oporomotĩva
15242,vacilante,,adj. va,Vava
14337,sobriedad,,f.,Ka’u’ỹ
13580,rugido,,m,py’ambu
12567,red,,ra,pira ñuhã
12589,redondear,,tr.,Mboapu’a


In [13]:
# --- Normalización granular y desdoblamiento de traducciones múltiples (español-guaraní) ---
def split_multiple_translations_eg(row):
    """
    Divide las traducciones separadas por números (2. 3. 4. ...) en líneas independientes,
    manteniendo la palabra_espanol y extrayendo tipo_etiqueta y categoria_gramatical para cada una.
    """
    palabra = row['palabra'] if 'palabra' in row else row['palabra_espanol']
    output = str(row['traduccion']) if 'traduccion' in row else str(row['output'])
    output = output if not pd.isnull(output) else ''
    partes = re.split(r'(?<=\.)\s*(\d+\.)\s*', output)
    traducciones = []
    if partes:
        traducciones.append(partes[0].strip())
        for i in range(1, len(partes), 2):
            traducciones.append(partes[i+1].strip() if i+1 < len(partes) else '')
    else:
        traducciones = [output.strip()]
    filas = []
    for trad in traducciones:
        tipo_etiqueta = ''
        tipo_match = re.match(r'^(neol\.|arc\.|h\.)', trad)
        if tipo_match:
            tipo_etiqueta = tipo_match.group(1)
            trad = trad[len(tipo_etiqueta):].lstrip('. ').strip()
        categoria_gramatical = ''
        cat_match = re.match(r'^(adj\.|tr\.|adv\.|prep\.|s\.|v\.|participio pasivo|participio activo|verbo transitivo|verbo intransitivo|adjetivo|sustantivo|pron\.|conj\.|interj\.|art\.|num\.|part\.|pref\.|suf\.)', trad)
        if cat_match:
            categoria_gramatical = cat_match.group(1)
            trad = trad[len(categoria_gramatical):].lstrip('. ').strip()
        filas.append({
            'palabra_espanol': palabra,
            'tipo_etiqueta': tipo_etiqueta,
            'categoria_gramatical': categoria_gramatical,
            'traduccion_limpia': trad
        })
    return filas

# Usar el DataFrame df_entries_espanol_guarani ya en memoria
filas_expandidas_eg = []
for _, row in df_entries_espanol_guarani.iterrows():
    filas_expandidas_eg.extend(split_multiple_translations_eg(row))

df_estructurado_eg = pd.DataFrame(filas_expandidas_eg)

print('Ejemplo de diccionario estructurado español-guaraní (una traducción por línea):')
display(df_estructurado_eg.sample(10))
df_estructurado_eg.to_csv('archivos/diccionario_espanol_guarani_estructurado.tsv', sep='\t', index=False, encoding='utf-8')
print('Archivo estructurado guardado: archivos/diccionario_espanol_guarani_estructurado.tsv')


Ejemplo de diccionario estructurado español-guaraní (una traducción por línea):


,palabra_espanol,tipo_etiqueta,categoria_gramatical,traduccion_limpia
4982,lustre,,,"m. Mimbi, vera."
2440,disputa,,,f. Ñorairõ.
5368,microcosmos,,,m. Arapy michĩ.
9120,velar,,,"intr. Iko páype, hesa ke’ỹme."
6727,pluviómetro,,,m. Ama ra’ãha.
7731,rencor,,,"m. Py’aro, py’araku."
2431,disolvente,,,m. Mbohykuha.
7908,retobarse,,,"prnl. Ñembopochy, mbohovái."
9356,yelmo,,,m. Akãngaoratã.
7913,retorcijón,,,m. Tyerasy kutu.


Archivo estructurado guardado: archivos/diccionario_espanol_guarani_estructurado.tsv
